In [1]:
import soundfile as sf
from datasets import load_dataset
from transformers import Speech2Text2Processor, SpeechEncoderDecoderModel

In [2]:
model = SpeechEncoderDecoderModel.from_pretrained("facebook/s2t-wav2vec2-large-en-de")
processor = Speech2Text2Processor.from_pretrained("facebook/s2t-wav2vec2-large-en-de")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch


ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

In [10]:
inputs = processor(ds["speech"][0], sampling_rate=16_000, return_tensors="pt")
print(inputs['input_values'].shape)

generated_ids = model.generate(inputs=inputs["input_values"], attention_mask=inputs["attention_mask"])
print(generated_ids)


transcription = processor.batch_decode(generated_ids)
print(transcription)